In [ ]:
# Import necessary libraries
import pandas as pd
%pip install textblob
from textblob import TextBlob
# Load datasets
# Load the datasets
try:
    words_freq = pd.read_csv("data/words_freq.csv")
    tweets = pd.read_csv("data/tweeets.csv")
    print(f"Words dataset shape: {words_freq.shape}")
    print(f"Tweets dataset shape: {tweets.shape}")
except FileNotFoundError as e:
    print(f"Error: {e}. Ensure the file paths are correct.")
    exit()

# Input Wordle word for sentiment analysis
word = input("Enter a Wordle word for sentiment analysis (5 letters): ").lower()

# Validate the word
if not word.isalpha() or len(word) != 5:
    print("Error: Please enter a valid 5-letter word using alphabetic characters.")
    exit()
                                                                                  
# Check if the word exists in the words_freq dataset
word_entry = words_freq[words_freq["word"].str.lower() == word]

if word_entry.empty:
    print(f"The word '{word}' was not found in the dataset.")
    exit()

# Get Wordle day and filter tweets
wordle_day = int(word_entry.iloc[0]["day"])
wordle_tweets = tweets[tweets["wordle_id"] == wordle_day]

if wordle_tweets.empty:
    print(f"No tweets found for Wordle #{wordle_day}.")
    exit()

# Analyze sentiment
print("Analyzing tweets...")
sentiments = {"positive": 0, "neutral": 0, "negative": 0}
polarity_scores = []

Note: you may need to restart the kernel to use updated packages.
Words dataset shape: (12972, 3)
Tweets dataset shape: (1178454, 5)
Analyzing tweets...

Results for 'cacao' (Wordle #373):
Total Tweets Analyzed: 341
Average Sentiment: 😊 Positive (0.009)
Sentiment Breakdown: {'positive': 53, 'neutral': 239, 'negative': 49}
